In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/riiid-test-answer-prediction/example_sample_submission.csv
/kaggle/input/riiid-test-answer-prediction/example_test.csv
/kaggle/input/riiid-test-answer-prediction/questions.csv
/kaggle/input/riiid-test-answer-prediction/train.csv
/kaggle/input/riiid-test-answer-prediction/lectures.csv
/kaggle/input/riiid-test-answer-prediction/riiideducation/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/riiid-test-answer-prediction/riiideducation/__init__.py


In [4]:
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter('ignore')

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go


import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, learning_curve
from sklearn.utils import shuffle
import lightgbm as lgb
from lightgbm import LGBMClassifier
import eli5

from catboost import CatBoostClassifier
import xgboost as xgb

import riiideducation
import riiideducation
import riiideducation
import riiideducation

%matplotlib inline
# for heatmap and other plots
colorMap1 = sns.color_palette("RdBu_r")
# for countplot and others plots
colorMap2 = 'Blues_r'

ModuleNotFoundError: No module named 'riiideducation'

In [ ]:
types = {
        'row_id': 'int64', 
        'timestamp': 'int64', 
        'user_id': 'int32', 
        'content_id': 'int16', 
        'content_type_id': 'int8',
        'task_container_id': 'int16', 
        'user_answer': 'int8', 
        'answered_correctly': 'int8', 
        'prior_question_elapsed_time': 'float32', 
        'prior_question_had_explanation': 'boolean'
}

In [ ]:
train = pd.read_csv(
    '/kaggle/input/riiid-test-answer-prediction/train.csv', 
    low_memory=False, 
    nrows=10**6, 
    dtype=types
)
questions=pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv')
lectures=pd.read_csv('/kaggle/input/riiid-test-answer-prediction/lectures.csv')
test=pd.read_csv('/kaggle/input/riiid-test-answer-prediction/example_test.csv')

train.head()

**DATA EXPLORATION & EDA**

row_id: (int64) ID code for the row.

timestamp: (int64) the time in milliseconds between this user interaction and the first event completion from that user.

user_id: (int32) ID code for the user.

content_id: (int16) ID code for the user interaction

content_type_id: (int8) 0 if the event was a question being posed to the user, 1 if the event was the user watching a lecture.

task_container_id: (int16) Id code for the batch of questions or lectures. For example, a user might see three questions in a row before seeing the explanations for any of them. Those three would all share a task_container_id.

user_answer: (int8) the user's answer to the question, if any. Read -1 as null, for lectures.

answered_correctly: (int8) if the user responded correctly. Read -1 as null, for lectures.

prior_question_elapsed_time: (float32) The average time in milliseconds it took a user to answer each question in the previous question bundle, ignoring any lectures in between. Is null for a user's first question bundle or lecture. Note that the time is the average time a user took to solve each question in the previous bundle.

prior_question_had_explanation: (bool) Whether or not the user saw an explanation and the correct response(s) after answering the previous question bundle, ignoring any lectures in between. The value is shared across a single question bundle, and is null for a user's first question bundle or lecture. Typically the first several questions a user sees were part of an onboarding diagnostic test where they did not get any feedback

In [ ]:
print(f"Train shape: {train.shape}")

In [ ]:
train.head()

In [ ]:
train.describe().style.background_gradient(cmap='Blues')

In [ ]:
print(f'Number of unique users: {len(np.unique(train.user_id))}')

Lets look at the missing data

In [ ]:
print(train.isnull().sum()/len(train))

Also let's check a correlation matrix to get more information between the columns

In [ ]:
corr_matrix=train.corr()
corr_matrix['answered_correctly'].sort_values(ascending=True)

In [5]:
plt.figure(figsize=(13,10))
sns.heatmap(corr_matrix,annot=True,
           linewidths=5,cmap=colorMap1)

NameError: name 'corr_matrix' is not defined

<Figure size 936x720 with 0 Axes>

Let's check the distribution of prior_question_elapsed_time

In [ ]:
plt.figure(figsize=(15, 10))
ax = sns.countplot(x="prior_question_elapsed_time", 
                   data=train[train['prior_question_elapsed_time'].notnull()],
                   palette=colorMap2)